In [1]:
import numpy as np
from qutip.measurement import measure
from qutip import (
    basis, Qobj, ket2dm
)


In [ ]:
from qutip import ket2dm, qeye

def depolarize_manual(rho, p):
    d = rho.shape[0]  # dimension (should be 2)
    return (1 - p) * rho + p * (qeye(d) / d)


In [ ]:
def generate_alice_data(n_qubits):
    alice_bits = np.random.randint(2, size=n_qubits)
    alice_bases = np.random.randint(2, size=n_qubits)
    return alice_bits, alice_bases


In [ ]:
def generate_bob_bases(n_qubits):
    return np.random.randint(2, size=n_qubits)


In [ ]:
from qutip import sigmax
from qutip.qip.operations.gates import hadamard_transform

def prepare_qubit_state(alice_bit, alice_basis):
    # Start in |0>
    state = basis(2, 0)  # |0>
    

    # If bit == 1, apply X
    if alice_bit == 1:
        state = sigmax() * state  # X|0> = |1>

    # If basis == 1, apply H
    if alice_basis == 1:
        # QuTiP's hadamard_transform() is a 2x2 matrix for single qubit
        H = hadamard_transform(1)  # 1 qubit Hadamard
        state = H * state

    return state


In [ ]:
def measure_qubit_state(state, bob_basis):
    # Z-basis projectors
    P0_z = ket2dm(basis(2, 0))  # |0><0|
    P1_z = ket2dm(basis(2, 1))  # |1><1|

    # X-basis states
    plus = (basis(2, 0) + basis(2, 1)).unit()   # |+> = (|0> + |1>)/sqrt(2)
    minus = (basis(2, 0) - basis(2, 1)).unit()  # |-> = (|0> - |1>)/sqrt(2)

    P0_x = ket2dm(plus)
    P1_x = ket2dm(minus)

    if bob_basis == 0:
        # Measure in Z
        M = [P0_z, P1_z]
    else:
        # Measure in X
        M = [P0_x, P1_x]

    # measure() gives (result_index, post_measurement_state)
    result_index, _ = measure(state, M)
    return result_index  # 0 or 1


In [ ]:
def compute_qber(alice_bits, alice_bases, bob_bits, bob_bases):
    matching_indices = [i for i in range(len(alice_bits)) if alice_bases[i] == bob_bases[i]]
    if len(matching_indices) == 0:
        return None  # no matching rounds -> invalid scenario

    mismatches = sum(alice_bits[i] != bob_bits[i] for i in matching_indices)
    return mismatches / len(matching_indices)


In [ ]:
def run_bb84_qutip(n_qubits=50):
    alice_bits, alice_bases = generate_alice_data(n_qubits)
    bob_bases = generate_bob_bases(n_qubits)

    bob_measurements = []
    for i in range(n_qubits):
        state = prepare_qubit_state(alice_bits[i], alice_bases[i])
        rho = ket2dm(state)
        noisy_rho = depolarize_manual(rho, p=0.2)  # Try p=0.01 to 0.2

        m = measure_qubit_state(noisy_rho, bob_bases[i])
        bob_measurements.append(m)


    # 4. Compute QBER
    qber = compute_qber(alice_bits, alice_bases, bob_measurements, bob_bases)
    print(f"BB84 (QuTiP) with {n_qubits} qubits -> QBER = {qber:.3f}")

    return qber


In [40]:
if __name__ == "__main__":
    qber_val = run_bb84_qutip(n_qubits=127)



BB84 (QuTiP) with 127 qubits -> QBER = 0.118
